# Gradient descent (optimization algorithm)

First-order optimization algorithm used to minimize (or maximize) functions by iteratively moving in the direction of the steepest descent.

- First-order because it uses first derivatives or gradients.
- The direction is determined by the negative of the gradient.
- Widely used in machine learning to optimize model parameters; specifically, it is fundamental to training many types of neural networks.

From a starting point, **Gradient Descent** takes (small) steps toward the function's minimum (maximum). 

![gradient descent](images/gd.png)


In this notebook, we will code it and explore its use.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

We need to define the following core *concepts of the optimization problem*:
- **Objective function** ($f$): the function we want to optimize
- **Gradient** ($\nabla\!f$): vector of partial derivatives for each dimension of the objective function

For example, let's define the function to optimize

$$f(x,y) = x^2+y^2$$

Optimize, in this setting, means to find the point $(x,y)$ that minimizes the value of $f$. We know that, in this illustrative function $f$, the point $(0,0)$ is the minimum.


In [ ]:
# Define the function f(x, y) = x^2 + y^2
def f(x, y):
    return x**2 + y**2

To calculate the gradient involves taking the **partial derivatives** of the function with respect to each dimension:
$$\nabla\!f(x,y)=\Big(\frac{\partial f}{\partial x},\frac{\partial f}{\partial y}\Big)$$

- For the above function, the partial derivative of the first dimension ($x$) is:
$$\frac{\partial f}{\partial x}=2*x$$
- and, for the second dimension ($y$):
$$\frac{\partial f}{\partial y}=2*y$$

A partial derivative for any dimension can be understood as the *rate of change* of the function in the direction of the dimension. Combining the partial derivatives for all the dimensions, the **gradient** forms a vector or direction in the function's space. It is important to note that it *points in the opposite direction of the minimum* ("uphill")

In [ ]:
# Define the gradient of f(x, y) = x^2 + y^2
def gradient(x, y):
    return np.array([2 * x, 2 * y])

Once we have defined the function and calculated its gradient, **gradient descent** involves following a simple procedure: given an initial point, repeatedly applying an update rule until the minimum point is reached. The key elements of the procedure are:
- The initial or **starting point**, random or user-defined.
- The **update rule**, a step in the opposite direction to the gradient (negative gradient).
- The **learning rate**, aka the scale for the update.

The **update rule** is just:

$$x_{new} = x_{old} - \alpha \cdot \frac{\partial f}{\partial x}(x_{old},y_{old})$$
$$y_{new} = y_{old} - \alpha \cdot \frac{\partial f}{\partial y}​(x_{old},y_{old})$$

That is, given a previous point $(x_{old},y_{old})$, a new point is obtained by *subtracting* from it the *scaled* gradient *evaluated* at the same point, $\nabla\!f(x_{old},y_{old})$:
- *evaluating* the gradient at $(x_{old},y_{old})$ gives not only the direction but also the magnitude of the step,
- *subtracting* because we need to move in the opposite direction of the gradient,
- *scaled* by $\alpha$ to control the rate of convergence.

This process is repeated iteratively (the new *old* is the previous *new* point) until the point reaches the minimum point of the function (the gradient is evaluated to $\sim 0$).

In [ ]:
# Gradient Descent function
# We save the whole path (all the intermediate points) for visualization
def gradient_descent(start_point, learning_rate, max_iterations):
    old_point = np.array(start_point, dtype=float)
    path = [old_point.copy()]  # to store the path of (x, y) values

    i = 0
    converged=False
    while not converged and i < max_iterations:
        grad_eval = gradient(old_point[0], old_point[1])
        new_point = #### YOUR CODE HERE ####
        path.append(new_point.copy())
        old_point = new_point
        converged = np.all(np.abs(grad_eval) < 10-9)
        i+=1

    return np.array(path)

Now, we have everything we need. Let's run an example:

In [ ]:
# Parameters
start_point = [-7, 7] # (x, y)
learning_rate = 0.1
iterations = 100

# Run gradient descent
path = gradient_descent(start_point, learning_rate, iterations)

In [ ]:
# Function that plots the objective function and the path of gradient descent
def plot_gd(func, grad, path, xlimits=[-10,10],ylimits=[-10,10], vp=None):
    # Configuration of the surface to show
    x = np.linspace(xlimits[0], xlimits[1], 100)
    y = np.linspace(ylimits[0], ylimits[1], 100)
    X, Y = np.meshgrid(x, y)
    Z = np.array([func(xi, yi) for xi, yi in zip(X.ravel(), Y.ravel())])
    Z = Z.reshape(X.shape)  # Reshape to match X and Y grid shapes
    #Z = func(X, Y)
    z_path = np.array([func(xi, yi) for xi, yi in zip(path[:, 0], path[:, 1])])
    
    fig = plt.figure(figsize=(10, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_surface(X, Y, Z, cmap='viridis', alpha=0.5)
    ax.plot(path[:, 0], path[:, 1], z_path, color='red', marker='o', markersize=5, label="Path towards minimum")
    
    gr_unit=grad(path[0, 0], path[0, 1])
    gr_unit/=np.sqrt(np.sum(gr_unit**2))
    ax.quiver(
            path[0, 0], path[0, 1], func(path[0, 0], path[0, 1]), # <-- starting point of vector
            gr_unit[0], gr_unit[1], 
            func(path[0, 0]+gr_unit[0], path[0, 1]+gr_unit[1])-func(path[0, 0], path[0, 1]), # <-- directions of vector
            color = 'blue', alpha = .8, arrow_length_ratio = 0.1, label="Unit vector of gradient at origin")
    ax.set_xlabel("x")
    ax.set_ylabel("y")
    ax.set_zlabel("f(x, y)")
    if vp is not None:
        ax.view_init(**vp) #change viewpoint
    plt.legend()
    plt.show()

In [ ]:
plot_gd(f, gradient, path)

## To understand the procedure, let's go step by step.

The selected **starting point** was $(x, y) = (-7, 7)$ and the learning rate $\alpha = 0.1$.

As we enter the procedure, the "old" point is the starting point

- Iteration 1:
    + Old point: $(x, y) = (-7, 7)$
    + Gradient evaluation: $\nabla f(-7, 7) = (2 \cdot -7, 2 \cdot 7) = (-14, 14)$
    + Update:
  $$
  x_{\text{new}} = x_{\text{old}} - \alpha \cdot \frac{\partial f}{\partial x}(x_{old},y_{old}) = -7 - 0.1 \cdot (-14) = -7 + 1.4 = -5.6
  $$
  $$
  y_{\text{new}} = y_{\text{old}} - \alpha \cdot \frac{\partial f}{\partial y}(x_{old},y_{old}) = 7 - 0.1 \cdot (14) = 7 - 1.4 = 5.6
  $$
    + New point: $(x, y) = (-5.6, 5.6)$
 
- Iteration 2:
    + Old point: $(x, y) = (-5.6, 5.6)$
    + Gradient evaluation: $\nabla f(-5.6, 5.6) = (2 \cdot -5.6, 2 \cdot 5.6) = (-11.2, 11.2)$
    + Update:
  $$
  x_{\text{new}} = -5.6 - 0.1 \cdot (-11.2) = -5.6 + 1.12 = -4.48
  $$
  $$
  y_{\text{new}} = 5.6 - 0.1 \cdot (11.2) = 5.6 - 1.12 = 4.48
  $$
    + New point: $(x, y) = (-4.48, 4.48)$

- Iteration 3:
    + Old point: $(x, y) = (-4.48, 4.48)$
    + Gradient evaluation: $\nabla f(-4.48, 4.48) = (2 \cdot -4.48, 2 \cdot 4.48) = (-8.96, 8.96)$
    + Update:
  $$
  x_{\text{new}} = -4.48 - 0.1 \cdot (-8.96) = -4.48 + 0.896 = -3.584
  $$
  $$
  y_{\text{new}} = 4.48 - 0.1 \cdot (8.96) = 4.48 - 0.896 = 3.584
  $$
    + New point: $(x, y) = (-3.584, 3.584)$

Each iteration produces a point closer to \((0, 0)\), where the objective function $f(x, y) = x^2 + y^2$ has its minimum. As empirically shown above, by continuing this procedure, gradient descent will eventually converge near $(0, 0)$, *if the learning rate is well-chosen*.


## The impact of choosing an appropriate learning rate
The learning rate scales the update given by the gradient's evaluation. It controls how large each update step is. If $\alpha$ is large, the algorithm might overshoot the minimum. If it is too small, convergence will be too slow. 

Let's see the previous example with different learning rates:

In [ ]:
# Parameters
#start_point = [-7, 7] # (x, y)
learning_rate = 0.95
#iterations = 100

# Run gradient descent
path = gradient_descent(start_point, learning_rate, iterations)
plot_gd(f, gradient, path, vp={"elev":15, "azim":60})

In [ ]:
# Parameters
#start_point = [-7, 7] # (x, y)
learning_rate = 0.01
#iterations = 100

# Run gradient descent
path = gradient_descent(start_point, learning_rate, iterations)
plot_gd(f, gradient, path, vp={"elev":15, "azim":60})

# Relationship with machine learning

The relationship between this optimization technique and machine learning might still be a little bit obscure. What is a "point" in my machine learning problem? what is the objective function? Let's try to clarify it, but it requires a change in our mindset.

- In ML, we usually work with models (functions) that have a set of parameters (the ones we need to learn!). These models, given an input, provide an output.

  For example: a simple regression model with parameters $(a,b)$:
  $$y=a\cdot x +b$$
  We first fix the parameters' values (we usually learn them). For example, $a=2.5$ and $b=1.5$.
  
  Then, given a value for $x$, the model computes and returns a value for $y$.

In [ ]:
def model(x, params={"a":2.5,"b":1.5}):
    return x*params["a"]+params["b"]

x = np.linspace(0, 10, 50)
y = model(x)

plt.figure(figsize=(7, 5))
plt.plot(x, y, color='green', linestyle='--', label='Linear regression model with (a,b)=(2.5,1.5)')
plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## Loss function: objective function of gradient descent

However, the story explained above is rarely the case in ML practice. 

1. We usually have a dataset of $N$ cases $(x,y)$ which are (noisy) samples of the model.

In [ ]:
np.random.seed(31) # fix the seed for reproducibility
N = 6
# Let's simulate a dataset with x-values between 0 and 10
x = np.random.random(N)*10
# and the corresponding observed y-values are the true evaluation of the function, plus some noise
y = model(x) + np.random.normal(0, 2, size=N)
dataset = np.array((x,y)).T
dataset

2. We decide the **type of model** that we want to learn: a linear regression model.
  
   *Note that* we usually do *not know the real model* and, therefore, its type.

3. We **find the parameters**, for the type of model chosen, that better match up the observed samples (data).

   This step, the actual **learning step**, involves several decisions:

   1. How do we decide which set of parameters matches the data better?
   2. How do find that *best* set of parameters?

  
To answer question 3.B, our procedure could be **gradient descent**.
   

To answer question 3.A, we usually define some sort of metric that measures the difference between the observed data and the expected answer of the model configured with a specific set of parameters. This is called the **loss function** and it will be our objective function: find the set of parameters that minimize the loss function.

A common loss function in regression problems is the **mean squared error**. 
$$
\text{loss} = \frac{1}{N} \sum_{i=1}^{N} (y_i - \hat{y}_i)^2
$$
where $y_i$ is the observed $y$-value for the i-th case of the dataset, and $\hat{y}_i$ is the corresponding prediction of the model. 

*Note that* we can define the loss as a function of the model's parameters:

In [ ]:
import autograd.numpy as anp  # you will understand this later on

def loss_function(a,b,data):
#    return anp.mean((data[:,1]-model(data[:,0], params={"a":a,"b":b}))**2)# we could call the model
    return anp.mean((data[:,1]-(data[:,0]*a+b))**2)# for efficiency, we just implement the model function here

In [ ]:
loss_function(3,4,dataset)

In fact, as the dataset is fixed in this optimization problem, we can define the objective function as only dependent on the model's parameters:

In [ ]:
f = lambda a, b : loss_function(a, b, dataset)

In [ ]:
f(3,4)

Now, we can apply gradient descent from a given starting point to find the point $(a,b)$ that minimizes $f$.

We just need to calculate the gradient:

1. Substitute $\hat{y}_i$ by $a \cdot x_i + b $ (model definition) into the loss function:
$$
\text{loss} = \frac{1}{N} \sum_{i=1}^{N} (y_i - (a \cdot x_i + b))^2
$$

2. The partial derivative with respect to $a$ is:
$$
\frac{\partial \, \text{loss}}{\partial a} = \frac{1}{N} \sum_{i=1}^{N} 2 \cdot (y_i - (a \cdot x_i + b)) \cdot (-x_i)
=
%\frac{\partial \, \text{loss}}{\partial a} = 
-\frac{2}{N} \sum_{i=1}^{N} x_i \cdot (y_i - (a \cdot x_i + b))
$$

3. The partial derivative with respect to $b$ is:

$$
\frac{\partial \, \text{loss}}{\partial b} = \frac{1}{N} \sum_{i=1}^{N} 2 \cdot (y_i - (a \cdot x_i + b)) \cdot (-1)
%\frac{\partial \, \text{loss}}{\partial b} 
= -\frac{2}{N} \sum_{i=1}^{N} (y_i - (a \cdot x_i + b))
$$
So, now, we can write the gradient as:

In [ ]:
# Define the gradient of f(a,b) = sum_1..N (yi - xi*a - b)^2
def gradient_loss(a, b, data):
    p_a = -2/data.shape[0]*np.sum(data[:,0]*(data[:,1]-(a*data[:,0]+b)))
    p_b = #### YOUR CODE HERE ####
    return np.array([p_a, p_b])

or

In [ ]:
gradient = lambda a, b : gradient_loss(a, b, dataset)

In [ ]:
gradient(3,4)

Lucky enough, you do not need to manually calculate the derivatives and implement them. This can also be automatically done with automatic differentiation libraries like `autograd`:

In [ ]:
from autograd import grad    # The only autograd function you may ever need

autogradient = grad(f,[0,1])       # Obtain its gradient function

In [ ]:
autogradient(3.,4.)

Now we can run **gradient descent** for this problem:

In [ ]:
# Parameters
start_point = [4, 3] # (a, b)
learning_rate = 0.015 # carefully chosen, try 0.01, 0.015, 0.02, 0.03
iterations = 200

# Run gradient descent
path = gradient_descent(start_point, learning_rate, iterations)
plot_gd(f, gradient, path, xlimits=[1,4],ylimits=[0,5], vp={"elev":15, "azim":110})

We can also plot the models with the different parameters throught the path of gradient descent:

In [ ]:
x = np.linspace(0, 10, 50)
nlinies = 5

plt.figure(figsize=(7, 5))

plt.scatter(dataset[:,0],dataset[:,1])

y = model(x)
plt.plot(x, y, color='green', linestyle='--', label='Real model with (a,b)=(2.5,1.5)')

for i in np.linspace(0, path.shape[0] - 1, nlinies + 2, dtype=int):
    y = model(x,{"a":path[i,0],"b":path[i,1]})
    plt.plot(x, y, color='blue', alpha=0.1, linestyle='-', 
             label='It. '+str(i)+' with (a,b)=('+f"{path[i,0]:.{2}f}"+","+f"{path[i,1]:.{2}f}"+')')

plt.xlabel('x')
plt.ylabel('y')
plt.legend()
plt.show()

## Questions:
- How do we choose the learning rate? Should it always be constant?
- What if the problem has more than one minimum? What do you know about saddle points?
- Implement **Stochastic Gradient Descent**